In [1]:
import os
import shutil
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
from pathlib import Path
import random

In [2]:
## Creating the folder with symbolic link image to save space, thus you must authorize the folder access for 
## cocodata directory in Scene Selector.
dataDir='/Volumes/Data/cocodata'
dataVal='val2017'
annVal='{}/annotations/instances_{}.json'.format(dataDir,dataVal)
dataTrain='train2017'
annTrain='{}/annotations/instances_{}.json'.format(dataDir,dataTrain)
exportTrainDir='/Volumes/AV_Workspace/export/train'
exportValDir='/Volumes/AV_Workspace/export/val'
if not os.path.exists(exportTrainDir):
    os.makedirs(exportTrainDir)
if not os.path.exists(exportValDir):
    os.makedirs(exportValDir)

In [3]:
# initialize COCO api for instance annotations
cocoVal=COCO(annVal)
cocoTrain=COCO(annTrain)

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
loading annotations into memory...
Done (t=14.45s)
creating index...
index created!


In [4]:
catsVal = cocoVal.loadCats(cocoVal.getCatIds())
catsTrain = cocoTrain.loadCats(cocoTrain.getCatIds())

In [5]:
def getCatName(catID):
    which = next(i for i,x in enumerate(catsTrain) if x['id']==catID)
    name = catsTrain[which]["name"]
    return name



In [6]:

nms=[cat['name'] for cat in catsVal]
print('COCO categories: \n{}\n'.format(' '.join(nms)))
nms = set([cat['supercategory'] for cat in catsVal])
print('COCO supercategories: \n{}'.format(' '.join(nms)))


COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
indoor sports kitchen vehicle accessory animal electronic appliance food furniture person outdoor


In [7]:
# get all images containing given categories, select one at random
imageIdSet=set()
catNms=['dog','cat','person','bicycle','car','motocycle','bus','ball','cup','tv','laptop','pizza']
catIds = cocoTrain.getCatIds(catNms=catNms);
imgIds = cocoTrain.getImgIds(catIds=catIds);
#print(catIds, imgIds)
#imgIds = coco.getImgIds(imgIds = [324158])
#img = cocoTrain.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
for catId in catIds:
    thisImgIds = cocoTrain.getImgIds(catIds=catId)
    random.shuffle(thisImgIds)
    thisImgIds = thisImgIds[:1500]
    #print(len(thisImgIds))
    imageIdSet.update(thisImgIds)
print(len(imageIdSet), len(imgIds))

13974 0


In [8]:
import xml.etree.ElementTree as ET
import ipywidgets as widgets
from IPython.display import display
from decimal import Decimal
from shutil import copyfile
import xml.etree.ElementTree as ET

out = widgets.Output(layout={'border': '1px solid black'})
twoinplaces = Decimal('0.01')
display(out)

for index,imageId in enumerate(imageIdSet):
    img = cocoTrain.loadImgs([imageId])
    imgName = img[0]['file_name']
    imgPath = '%s/images/%s/%s'%(dataDir,dataTrain,imgName)
    out.clear_output(wait=True)
    with out:
        #image.show()
        print(f'process image number: {imgName}, progress: {Decimal((index + 1) / len(imageIdSet) * 100).quantize(twoinplaces)}%')
    root = ET.Element('annotation')
    filenameElement = ET.Element('filename',{})
    filenameElement.text = imgName
    root.append(filenameElement)
    ##not sure if retrieving image size from Scene Selector will be faster
    image = io.imread(imgPath)
    sizeElement = ET.Element('size')
    widthElement = ET.Element('width')
    widthElement.text = str(image.shape[1])
    heightElement = ET.Element('height')
    heightElement.text = str(image.shape[0])
    sizeElement.append(widthElement)
    sizeElement.append(heightElement)
    root.append(sizeElement)
    
    targetPath = '{}/{}'.format(exportTrainDir,imgName)
    if os.path.exists(targetPath):
        os.remove(targetPath)
    #os.symlink(imgPath,targetPath)
    shutil.copy(imgPath,targetPath)
    annIds = cocoTrain.getAnnIds(imgIds=[imageId])
    anns = cocoTrain.loadAnns(annIds)
    
    for ann in anns:
        objectElement = ET.Element('object')
        thisCatId = ann["category_id"]
        if thisCatId in catIds:
            name = getCatName(thisCatId)
            nameElement = ET.Element('name')
            nameElement.text = name
            objectElement.append(nameElement)
            bboxElement = ET.Element('bndbox')
            bbox = ann['bbox']
            #print(f' bounding box: {bbox}, this cat: {thisCatId}')
            #coco use top/left coordinate, turicreate needs center
            xminElement = ET.Element('xmin')
            yminElement = ET.Element('ymin')
            xmaxElement = ET.Element('xmax')
            ymaxElement = ET.Element('ymax')
            xminElement.text = str(bbox[0])
            yminElement.text = str(bbox[1])
            xmaxElement.text = str(bbox[0] + bbox[2])
            ymaxElement.text = str(bbox[1] + bbox[3])
            bboxElement.extend([xminElement,xmaxElement,yminElement,ymaxElement])
            objectElement.append(bboxElement)
            root.append(objectElement)
    if (len(root.findall('object')) > 0):
        imgNameStem = Path(imgName).stem
        tree = ET.ElementTree(element=root)
        tree.write('{}/{}.annotations'.format(exportTrainDir,imgNameStem))
    


Output(layout=Layout(border='1px solid black'))

In [9]:
#validation
imageIdSet=set()
catNms=['dog','cat','person','bicycle','car','motocycle','bus','ball','cup','tv','laptop','pizza']
catIds = cocoVal.getCatIds(catNms=catNms);
imgIds = cocoVal.getImgIds(catIds=catIds);
#print(catIds, imgIds)
#imgIds = coco.getImgIds(imgIds = [324158])
#img = cocoTrain.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
for catId in catIds:
    thisImgIds = cocoVal.getImgIds(catIds=catId)
    random.shuffle(thisImgIds)
    thisImgIds = thisImgIds[:100]
    print(len(thisImgIds))
    imageIdSet.update(thisImgIds)

100
100
100
100
100
100
100
100
100
100


In [10]:
import xml.etree.ElementTree as ET
import ipywidgets as widgets
from IPython.display import display
from decimal import Decimal
from shutil import copyfile
import xml.etree.ElementTree as ET

out = widgets.Output(layout={'border': '1px solid black'})
twoinplaces = Decimal('0.01')
display(out)

for index,imageId in enumerate(imageIdSet):
    img = cocoVal.loadImgs([imageId])
    imgName = img[0]['file_name']
    imgPath = '%s/images/%s/%s'%(dataDir,dataVal,imgName)
    out.clear_output(wait=True)
    with out:
        #image.show()
        print(f'process image number: {imgName}, progress: {Decimal((index + 1) / len(imageIdSet) * 100).quantize(twoinplaces)}%')
    root = ET.Element('annotation')
    filenameElement = ET.Element('filename',{})
    filenameElement.text = imgName
    root.append(filenameElement)
    
    image = io.imread(imgPath)
    sizeElement = ET.Element('size')
    widthElement = ET.Element('width')
    widthElement.text = str(image.shape[1])
    heightElement = ET.Element('height')
    heightElement.text = str(image.shape[0])
    sizeElement.append(widthElement)
    sizeElement.append(heightElement)
 
    root.append(sizeElement)
    targetPath = '{}/{}'.format(exportValDir,imgName)
    if os.path.exists(targetPath):
        os.remove(targetPath)
    #os.symlink(imgPath,targetPath)
    shutil.copy(imgPath,targetPath)
    annIds = cocoVal.getAnnIds(imgIds=[imageId])
    anns = cocoVal.loadAnns(annIds)
    
    for ann in anns:
        objectElement = ET.Element('object')
        thisCatId = ann["category_id"]
        if thisCatId in catIds:
            name = getCatName(thisCatId)
            nameElement = ET.Element('name')
            nameElement.text = name
            objectElement.append(nameElement)
            bboxElement = ET.Element('bndbox')
            bbox = ann['bbox']
            #print(f' bounding box: {bbox}, this cat: {thisCatId}')
            #coco use top/left coordinate, turicreate needs center
            xminElement = ET.Element('xmin')
            yminElement = ET.Element('ymin')
            xmaxElement = ET.Element('xmax')
            ymaxElement = ET.Element('ymax')
            xminElement.text = str(bbox[0])
            yminElement.text = str(bbox[1])
            xmaxElement.text = str(bbox[0] + bbox[2])
            ymaxElement.text = str(bbox[1] + bbox[3])
            bboxElement.extend([xminElement,xmaxElement,yminElement,ymaxElement])
            objectElement.append(bboxElement)
            root.append(objectElement)
    if (len(root.findall('object')) > 0):
        imgNameStem = Path(imgName).stem
        tree = ET.ElementTree(element=root)
        tree.write('{}/{}.annotations'.format(exportValDir,imgNameStem))



Output(layout=Layout(border='1px solid black'))